In [1]:
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
import seaborn as sns

import harness

np.warnings.filterwarnings('ignore', 'Ill-conditioned matrix')

In [2]:
flights_train = pd.read_csv('data/flights_train.csv', index_col=0)
flights_test = pd.read_csv('data/flights_test.csv', index_col=0)

In [3]:
flights_clean = harness.clean_train(flights_train)

In [4]:
add_features = harness.chain(
    harness.add_date_parts, harness.add_haul, harness.add_weather
)

In [5]:
flights_with_features = add_features(flights_clean)

In [6]:
x_transform = harness.chain(
    add_features,
    [harness.add_all_grouped_stats, flights_with_features],
    harness.only_numeric
)

In [7]:
transformer = harness.DataTransformer(
    x_transform, harness.normalize, harness.unnormalize
)

In [8]:
x_train, y_train = transformer.extract_transform(flights_clean)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator PowerTransformer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [ ]:
def train_model():
    GBR = GradientBoostingRegressor(n_estimators=10, n_jobs=-1)
    grid_search = ms.GridSearchCV(
        GBR, {'max_depth' : [None, 2,3,5],'max_leaf_nodes' : [None, 3,5,10]}
    )
    grid_search.fit(x_train.values, y_train.values)
    harness.save(grid_search.best_estimator_, 'everything_linear_model')

In [ ]:
#train_model()

In [ ]:
GBR = harness.load('everything_linear_model')
GBR_model = harness.TrainedModel(GBR, transformer)

In [ ]:
GBR_model.validate(harness.clean_train(flights_test))

In [ ]:
GBR_model.submit(
    'data/test.csv', 'time_of_day_submission.csv', 'predicted_delay'
)